In [1]:
import pandas as pd
import os
import random
import torch
import pickle

In [2]:
random.seed(2024)
!ls

data		  preprocess_amazon.ipynb  Statistics_KuaiRec.ipynb
item_class.ipynb  readme.md


# Load original dataset

### Selecting datasets

In [4]:

dataset_name =  "yelp"
root = 'data/'+dataset_name+'/'
path = root + dataset_name + '.inter'
dataset = pd.read_csv(path, header=0,sep='\t', skipinitialspace=True,usecols=[0,1,2,3])
# print(dataset.head(2))
# if dataset_name ==  "mind":
#     dataset=dataset.drop('label:float', axis=1)
dataset.head()

,user_id:token,item_id:token,rating:float,timestamp:float
0,bv2nCi5Qv5vroFiqKGopiw,0W4lkclzZThpx3V65bVgig,5,1464364800
1,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,1464364800
2,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,1464364800
3,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,1464364800
4,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,1464364800


In [78]:
# dataset_name =  "kuairec"
# root = 'datasets/data/'+dataset_name+'/'
# path=root + 'big_matrix.csv'
# print("Loading matrix...")
# dataset = pd.read_csv(path,usecols=[0,1,6,7])
# dataset.head()
# dataset.columns = ['user_id','item_id', 'timestamp','rating']
# dataset = dataset[dataset.rating > 2]

In [5]:

dataset.columns = ['user_id','item_id', 'rating','timestamp']
# dataset = dataset[dataset.rating > 3]
# dataset.columns = ['user_id', 'item_id',  'timestamp']
dataset.shape

(5261669, 4)

In [8]:
dataset_name =  "yelp"
root = 'data/'+dataset_name+'/'
path = root + dataset_name + '.item'
item = pd.read_csv(path, header=0,sep='\t', skipinitialspace=True,usecols=[0,3])
# print(dataset.head(2))
# if dataset_name ==  "mind":
#     dataset=dataset.drop('label:float', axis=1)
item.columns=['item_id','item_class']
print(item['item_class'].value_counts())
item.head(10)

item_class
Las Vegas                26775
Phoenix                  17213
Toronto                  17206
Charlotte                 8553
Scottsdale                8228
                         ...  
Middleburg Hts.              1
Midddleton                   1
Scarobrough                  1
Harrisbug                    1
Saint-Basile-Le-Grand        1
Name: count, Length: 1093, dtype: int64


,item_id,item_class
0,FYWN1wneV18bWNgQjJ2GNg,Ahwatukee
1,He-G7vWjzVUysIKrfNbPUQ,McMurray
2,KQPW8lFf1y5BT2MxiSZ3QA,Phoenix
3,8DShNS-LuFqpEWIp0HxijA,Tempe
4,PfOCPjBrlQAnz__NXj9h_w,Cuyahoga Falls
5,o9eMRCWt5PkpLDE0gOPtcQ,Stuttgart
6,kCoE3jvEtg6UVz5SOD3GVw,Las Vegas
7,OD2hnuuTJI9uotcKycxg1A,Las Vegas
8,EsMcGiZaQuG1OOvL9iUFug,Wexford
9,TGWhGNusxyMaA4kQVBNeew,Henderson


In [9]:
empty_classes = item[item['item_class'].isna() | (item['item_class'] == '')]
empty_classes

,item_id,item_class
146524,Cr3LJ2sKGNKEYIAjmG7aCA,NaN


In [82]:
# 关心的类别列表
categories = [
    'Action', 'Crime', 'Film-Noir', 'Musical', 'Sci-Fi', 'Adventure', 'Animation',
    'Biography', 'Comedy', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Game-Show',
    'History', 'Horror', 'Music', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Short',
    'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'
]
item['item_class'] = item['item_class'].str.split()
item['item_class'] = item['item_class'].apply(lambda x: [item for item in x if item in categories])
for category in categories:
    item[category] = item['item_class'].apply(lambda x: 1 if category in x else 0)
item.head()

,item_id,item_class,Action,Crime,Film-Noir,Musical,Sci-Fi,Adventure,Animation,Biography,...,Mystery,News,Reality-TV,Romance,Short,Sport,Talk-Show,Thriller,War,Western
0,1,"[Animation, Comedy]",0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,"[Action, Adventure, Thriller]",1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,[Thriller],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,"[Action, Comedy, Drama]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,"[Crime, Drama, Thriller]",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [83]:
item.drop(columns=['item_id','item_class']).sum()

Action         251
Crime          109
Film-Noir       24
Musical         56
Sci-Fi         101
Adventure      135
Animation       42
Biography        0
Comedy         505
Documentary     50
Drama          725
Family           0
Fantasy         22
Game-Show        0
History          0
Horror          92
Music            0
Mystery         61
News             0
Reality-TV       0
Romance        247
Short            0
Sport            0
Talk-Show        0
Thriller       251
War             71
Western         27
dtype: int64

In [84]:
item = item.drop(columns=['Biography'])

In [85]:
dataset = pd.merge(dataset, item, on='item_id', how='left')
# print(dataset['item_class'].value_counts())
dataset.head()

,user_id,item_id,rating,timestamp,item_class,Action,Crime,Film-Noir,Musical,Sci-Fi,...,Mystery,News,Reality-TV,Romance,Short,Sport,Talk-Show,Thriller,War,Western
0,196,242,3,881250949,[Comedy],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,186,302,3,891717742,"[Crime, Film-Noir, Mystery, Thriller]",0,1,1,0,0,...,1,0,0,0,0,0,0,1,0,0
2,22,377,1,878887116,[Comedy],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,244,51,2,880606923,"[Drama, Romance, War, Western]",0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
4,166,346,1,886397596,"[Crime, Drama]",0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
dataset = dataset[dataset.rating > 3]

### filter by number of interactions

In [102]:
user_threshold = 5
item_threshold = 5


filtered_dataset = dataset.copy()
while(True):
    ori_len = len(filtered_dataset)
    print(ori_len)
    filtered_dataset = filtered_dataset[filtered_dataset['user_id'].map(filtered_dataset['user_id'].value_counts()) >= user_threshold]
    filtered_dataset = filtered_dataset[filtered_dataset['item_id'].map(filtered_dataset['item_id'].value_counts()) >= item_threshold]
    print(len(filtered_dataset))
    # break
    if len(filtered_dataset) == ori_len:
        break
print('done!')
# filtered_dataset.head()

12195566
12178575
12178575
12178563
12178563
12178563
done!


# Map all ids

In [103]:

all_user = filtered_dataset.user_id
all_item = filtered_dataset.item_id

In [104]:
user_id, user_token = pd.factorize(all_user)
print("num user: ",len(user_token))
item_id, item_token = pd.factorize(all_item)
print("num item: ", len(item_token))
user_mapping_dict = {_: idx for idx, _ in enumerate(user_token)}
item_mapping_dict = {_: idx+1 for idx, _ in enumerate(item_token)}

num user:  137474
num item:  15405


In [105]:
mapped_dataset = filtered_dataset.copy()
mapped_dataset['user_id'] = filtered_dataset['user_id'].apply(lambda x: user_mapping_dict[x])
mapped_dataset['item_id'] = filtered_dataset['item_id'].apply(lambda x: item_mapping_dict[x])
mapped_dataset.head()

,user_id,item_id,rating,timestamp
0,0,1,3.5,1112486027
1,0,2,3.5,1112484676
2,0,3,3.5,1112484819
3,0,4,3.5,1112484727
4,0,5,3.5,1112484580


In [106]:
dataset = mapped_dataset.sort_values(by=['user_id', 'timestamp'])
item_class = dataset.drop(columns=['user_id','rating','timestamp','item_class'])
item_class = item_class.groupby('item_id').max().reset_index()
print(len(item_class))
# with open(os.path.join(root, 'item_class.pkl'), 'wb') as f:
#     pickle.dump(item_class, f)

KeyError: "['item_class'] not found in axis"

In [107]:
dataset = mapped_dataset.sort_values(by=['user_id', 'timestamp'])
user_group = dataset.groupby('user_id')['item_id'].apply(list)
user_group.head()

user_id
0    [20, 19, 81, 56, 23, 90, 14, 103, 7, 6, 54, 40...
1    [163, 169, 172, 177, 181, 165, 17, 43, 180, 18...
2    [17, 238, 65, 278, 279, 265, 266, 277, 205, 22...
3    [295, 297, 302, 167, 17, 300, 305, 296, 298, 2...
4    [163, 310, 329, 330, 327, 331, 334, 335, 311, ...
Name: item_id, dtype: object

In [108]:
k=0
for i in user_group:
    k+=len(i)
print(k)
print(k/len(user_group))

12178563
88.58811848058542


### In-domain sequences

In [109]:
# max_seq_len = 20
# 
# def truncate_or_pad(seq):
#     cur_seq_len = len(seq)
#     if cur_seq_len > max_seq_len:
#         return seq[-max_seq_len:], max_seq_len
#     else:
#         return seq + [PAD] * (max_seq_len - cur_seq_len), cur_seq_len

# 
# dataset = mapped_dataset.sort_values(by=['user_id', 'timestamp'])
# user_group = dataset.groupby('user_id')['item_id'].apply(list)
train, val, test = [], [], []

for user_id, user_seq in list(zip(user_group.index, user_group.tolist())):
    # iterate on each user sequence
    # ------ test sample ------------
    test_data = user_seq[-1]
    test.append([test_data])
    # ------ val sample -------------
    val_data = user_seq[-2]
    val.append([val_data])
    # ------ train sample -----------
    train_data= user_seq[:-2]
    train.append(train_data)
data = {'train': train, 'val': val, 'test': test}
with open(os.path.join(root, 'dataset.pkl'), 'wb') as f:
    pickle.dump(data, f)
# torch.save(train, os.path.join(output_path, 'dataset.pkl'))

print('done!')

done!


### mixed sequence

# Save

In [12]:
# for d_name, d in zip(output_dataset_name_list, mapped_dataset_list):
#     d.to_csv(os.path.join(output_path, d_name, 'inter.csv'), sep=',', index=None)